In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU, Bidirectional
from keras.optimizers import SGD

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from sklearn.metrics import precision_score, confusion_matrix

import math

import warnings
warnings.filterwarnings("ignore")

2024-12-05 09:33:17.177672: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 09:33:17.177718: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 09:33:17.177768: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 09:33:17.188655: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
data = pd.read_csv('dataset/stocknet-dataset-master/price/raw/NEE.csv')

In [4]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-09-04,67.339996,67.419998,66.930000,67.190002,57.616905,1763700
1,2012-09-05,67.220001,67.459999,67.129997,67.150002,57.582603,1371800
2,2012-09-06,65.989998,67.440002,65.949997,67.410004,57.805576,5098300
3,2012-09-07,67.529999,67.949997,67.389999,67.650002,58.011375,2474000
4,2012-09-10,67.879997,68.440002,67.769997,67.940002,58.260056,2375300
...,...,...,...,...,...,...,...
1253,2017-08-28,150.300003,150.899994,150.039993,150.880005,150.880005,874900
1254,2017-08-29,151.029999,151.279999,150.139999,150.720001,150.720001,1399300
1255,2017-08-30,150.419998,150.839996,150.169998,150.619995,150.619995,1061500
1256,2017-08-31,151.000000,151.020004,150.210007,150.509995,150.509995,1706500


In [5]:
train = data.iloc[:1000, 4:5]
test = data.iloc[1000:, 4:5]
test.reset_index(inplace=True)
del test['index']

In [6]:
sc = MinMaxScaler(feature_range=(0,1))
train_scaled = sc.fit_transform(train)

In [7]:
x_train = []
y_train = []

for i in range(60,1000):
    x_train.append(train_scaled[i-60:i, 0])
    y_train.append(train_scaled[i,0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [8]:
x_train[0]

array([0.01086963, 0.01024851, 0.01428581, 0.01801248, 0.02251559,
       0.01940994, 0.00962738, 0.03431675, 0.02018638, 0.0164597 ,
       0.01428581, 0.01583857, 0.02422368, 0.0332298 , 0.04658385,
       0.04813661, 0.05372669, 0.05667703, 0.05962739, 0.05683236,
       0.06335407, 0.06568328, 0.07127335, 0.06739137, 0.06506214,
       0.05931677, 0.05279505, 0.05217391, 0.04472057, 0.04922369,
       0.0621118 , 0.08214287, 0.08633548, 0.08121123, 0.0802796 ,
       0.06630441, 0.04720498, 0.05046584, 0.04704967, 0.05543478,
       0.05248455, 0.05031064, 0.04844725, 0.04130441, 0.02422368,
       0.01739135, 0.01381986, 0.00465843, 0.00947205, 0.00310565,
       0.        , 0.01335404, 0.01894411, 0.01894411, 0.01180127,
       0.01102483, 0.0228261 , 0.0206522 , 0.01428581, 0.02515533])

In [9]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [10]:
dataset_total = pd.concat((data['Close'][:1000], data['Close'][1000:]), axis=0)
print(dataset_total.shape)

inputs = dataset_total[len(dataset_total)-len(test)-60 : ].values
print(inputs.shape)
inputs = inputs.reshape(-1,1)
print(inputs.shape)
inputs = sc.transform(inputs)
print(inputs.shape)

(1258,)
(318,)
(318, 1)
(318, 1)


In [11]:
# regressorGRU = Sequential()
# # First GRU layer with Dropout regularisation
# regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(x_train.shape[1],1), activation='tanh'))
# regressorGRU.add(Dropout(0.2))
# # Second GRU layer
# regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(x_train.shape[1],1), activation='tanh'))
# regressorGRU.add(Dropout(0.2))
# # Third GRU layer
# regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(x_train.shape[1],1), activation='tanh'))
# regressorGRU.add(Dropout(0.2))
# # Fourth GRU layer
# regressorGRU.add(GRU(units=50, activation='tanh'))
# regressorGRU.add(Dropout(0.2))
# # The output layer
# regressorGRU.add(Dense(units=1))

In [12]:
import seaborn as sns

In [13]:
df_sentiment=pd.read_csv('sentiment/stock_data.csv')

In [14]:
def clean_text(Text):
    import re
    # Remove links
    Text = re.sub(r'http\S+|www\S+|https\S+', '', Text, flags=re.MULTILINE)
    # Remove mentions and hashtags
    Text = re.sub(r'\@w+|\#', '', Text)
    # Remove digits
    Text = re.sub(r'\d+', '', Text)
    # Convert to lowercase
    Text = Text.lower()
    # Remove stop words
    from nltk.corpus import stopwords
    Text = ' '.join(word for word in Text.split())
    return Text

df_sentiment['Text'] = df_sentiment['Text'].apply(clean_text)

In [15]:
df_sentiment['Text'] = df_sentiment['Text'].fillna('').astype(str)
df_sentiment['Text'] = df_sentiment['Text'].apply(clean_text)

In [16]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(df_sentiment['Text'],df_sentiment['Sentiment'],test_size=0.2,random_state=42)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x_train_tf = vectorizer.fit_transform(x_train)
x_test_tf = vectorizer.transform(x_test)

In [18]:
import pandas as pd
import json
from pandas import json_normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

vectorizer = TfidfVectorizer()
x_train_tf = vectorizer.fit_transform(x_train)

x_train_tf, x_val_tf, y_train, y_val = train_test_split(x_train_tf, y_train, test_size=0.2, random_state=42)

model = Sequential([
    Dense(128, activation='relu', input_dim=x_train_tf.shape[1]),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='tanh')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

x_train_tf_dense = x_train_tf.toarray()
x_val_tf_dense = x_val_tf.toarray()

model.fit(x_train_tf_dense, y_train, epochs=50, batch_size=32, validation_data=(x_val_tf_dense, y_val))

2024-12-05 09:33:21.141519: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1883] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31118 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:5e:00.0, compute capability: 7.0


Epoch 1/50


2024-12-05 09:33:24.285715: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7799c8a14da0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-05 09:33:24.285748: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM3-32GB, Compute Capability 7.0
2024-12-05 09:33:24.292010: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-05 09:33:24.312171: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-12-05 09:33:24.390389: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


116/116 [==============================] - 3s 6ms/step - loss: 0.8418 - mae: 0.8712 - val_loss: 0.6633 - val_mae: 0.7244
Epoch 2/50
116/116 [==============================] - 0s 3ms/step - loss: 0.4360 - mae: 0.4910 - val_loss: 0.6173 - val_mae: 0.5719
Epoch 3/50
116/116 [==============================] - 0s 4ms/step - loss: 0.1889 - mae: 0.2545 - val_loss: 0.6400 - val_mae: 0.5184
Epoch 4/50
116/116 [==============================] - 0s 3ms/step - loss: 0.0998 - mae: 0.1463 - val_loss: 0.6836 - val_mae: 0.5215
Epoch 5/50
116/116 [==============================] - 0s 4ms/step - loss: 0.0500 - mae: 0.0863 - val_loss: 0.6983 - val_mae: 0.5055
Epoch 6/50
116/116 [==============================] - 0s 3ms/step - loss: 0.0301 - mae: 0.0586 - val_loss: 0.7070 - val_mae: 0.4908
Epoch 7/50
116/116 [==============================] - 0s 3ms/step - loss: 0.0206 - mae: 0.0429 - val_loss: 0.7256 - val_mae: 0.4956
Epoch 8/50
116/116 [==============================] - 0s 3ms/step - loss: 0.0180 - mae:

In [19]:
import os
import json
import pandas as pd
from pandas import json_normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime

def clean_text(text):
    return text.lower()

# Define the folder path
folder_path = 'dataset/stocknet-dataset-master/tweet/raw/NEE/'

# Initialize an empty list to store results
results = []

# Generate a complete date range
start_date = datetime(2014, 1, 1)
end_date = datetime(2016, 12, 31)
all_dates = pd.date_range(start=start_date, end=end_date).to_frame(index=False, name='date')

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    try:
        file_date = datetime.strptime(filename.split('.')[0], '%Y-%m-%d').date()
    except ValueError:
        print(f"Skipping invalid filename: {filename}")
        continue  # Skip files that don't match the date format

    try:
        # Read JSON
        df_tweet = pd.read_csv(file_path, delimiter='\t', header=None, names=['json_content'])
        df_tweet['parsed'] = df_tweet['json_content'].apply(json.loads)
        df_normalized = json_normalize(df_tweet['parsed'])

        # Clean the text data
        df_normalized['text'] = df_normalized['text'].apply(clean_text)
        df_normalized['text'] = df_normalized['text'].fillna('').astype(str)

        # Vectorize tweets
        df_tf = vectorizer.transform(df_normalized['text'])

        df_tf_dense = df_tf.toarray()
        
        # Predict sentiment using the model
        y_pred = model.predict(df_tf_dense, verbose=0)
        y_pred_label = ((sum(y_pred > 0).astype(int) / len(y_pred)) - 0.5) * 2

        results.append({'date': file_date, 'sentiment_label': y_pred_label})

    except Exception as e:
        print(f"Error processing {filename}: {e}")

results_df = pd.DataFrame(results)

results_df['date'] = pd.to_datetime(results_df['date'])
merged_df = pd.merge(all_dates, results_df, on='date', how='left')

merged_df['sentiment_label'] = merged_df['sentiment_label'].fillna(0)

merged_df = merged_df.sort_values(by='date').reset_index(drop=True)

In [20]:
df = data
df['Date'] = pd.to_datetime(df['Date'])
merged_df['date'] = pd.to_datetime(merged_df['date'])

df = pd.merge(df, merged_df[['date', 'sentiment_label']], left_on='Date', right_on='date', how='left')

df = df.drop(columns=['date'])

# Days without tweets get sentiment value 0
df['sentiment_label'] = df['sentiment_label'].fillna(0)

In [21]:
# Drop redundant sentiment column
#df = df.drop(columns=['sentiment_label_y'])
#df.rename(columns={'sentiment_label_x': 'sentiment_label'}, inplace=True)

df['sentiment_label'] = df['sentiment_label'].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)

In [22]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, 
    precision_score, recall_score, f1_score
)
from sklearn.utils import class_weight

df['Date'] = pd.to_datetime(df['Date'])

# Create target
df['Target'] = (df['Close'] > df['Close'].shift(1)).astype(int)

# Drop empty rows
df = df.dropna()

features = ['Open', 'High', 'Low', 'Close', 'Volume', 'sentiment_label']
X = df[features].values
y = df['Target'].values

n_timesteps = 30

split_index = 1000
X_train_raw = X[:split_index]
X_test_raw = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_test_scaled = scaler.transform(X_test_raw)

# Create sequences using a sliding window
def create_sequences(features, target, timesteps):
    X_seq, y_seq = [], []
    for i in range(len(features) - timesteps):
        X_seq.append(features[i:i + timesteps])  # Past `timesteps` data
        y_seq.append(target[i + timesteps])  # Target at `timesteps + 1`
    return np.array(X_seq), np.array(y_seq)

x_train, y_train_seq = create_sequences(X_train_scaled, y_train, n_timesteps)
x_test, y_test_seq = create_sequences(X_test_scaled, y_test, n_timesteps)

# Weights to adjust for imbalances in the dataset
class_weights = class_weight.compute_class_weight(
    'balanced', classes=[0, 1], y=y_train_seq
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

def create_bidirectional_gru_model(input_shape):
    model = Sequential([
        Bidirectional(GRU(units=64, return_sequences=True, activation='tanh'), input_shape=input_shape),
        Dropout(0.2),
        BatchNormalization(),
        Bidirectional(GRU(units=32, return_sequences=False, activation='tanh')),
        Dropout(0.2),
        Dense(units=1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (x_train.shape[1], x_train.shape[2])
regressorGRU = create_gru_model(input_shape)

regressorGRU.fit(
    x_train, y_train_seq, 
    epochs=50, batch_size=64, 
    class_weight=class_weight_dict, verbose=0
)

# Predict
all_predictions = []
for i in range(len(x_test)):
    x_test_sample = x_test[i].reshape(1, x_test.shape[1], x_test.shape[2])
    prediction = regressorGRU.predict(x_test_sample, verbose=0)[0, 0]
    mean_pred = np.mean(all_predictions)
    std = np.std(all_predictions)
    all_predictions.append(1 if prediction >= (mean_pred + std) else 0)

all_predictions = np.array(all_predictions)
precision = precision_score(y_test_seq, all_predictions)
recall = recall_score(y_test_seq, all_predictions)
f1 = f1_score(y_test_seq, all_predictions)
accuracy = accuracy_score(y_test_seq, all_predictions)

# Print metrics
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Accuracy: {accuracy:.2f}")

print(confusion_matrix(y_test_seq, all_predictions))

NameError: name 'create_gru_model' is not defined